***CLAB 3****

Fist, what is condtional double sort?

Conditional Double Sort is a widely used empirical method in asset pricing to examine the relationship between stock characteristics and returns.

It normally firsly sorts stocks into quantiles based on the primary varialbe, and then, within each quantile, we further sorted stocks into quantiles based on the secondary variable.

*** In the case of Q3***
in our case, the primary variable is SIZE, and the second variable is Amihud Illiquidity.

which, first step, we sort all stock in 5 quintile based on size， and in each size-based quintile, further sort stocks based on Amihud Illiquidity. <font color="red">Overall, we will have 5 * 5=25 portfolio.</font>

<span style="font: bold 18px sans-serif; color:green">frist, we need the dataset that inlucdes firm size, amilhud factors and FF3 factors and capm factors</span>

**in the this example, We use U.S. monthly datafrom CRSP to obtained the fundamental data. Our sample period is from January 2018- December 2019.


In [57]:
pip install pandas numpy matplotlib statsmodels

Note: you may need to restart the kernel to use updated packages.


In [58]:
import pandas as pd

***1.Import daily data, which is for caluclating the month return and amihud illiquidility***

In [59]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\NZDATA-FORCALCULATINGAMIHUD.csv'
df = pd.read_csv(file_path)
df.head(5)

,Stock_Name,Date,TOT RETURN IND,TURNOVER BY VOLUME,UNADJUSTED PRICE
0,2 CHEAP CARS GROUP,25/02/2021 0:00,100.00,20.0,1.30
1,2 CHEAP CARS GROUP,26/02/2021 0:00,100.00,2.5,1.30
2,2 CHEAP CARS GROUP,3/03/2021 0:00,99.23,1.3,1.29
3,2 CHEAP CARS GROUP,4/03/2021 0:00,98.46,6.0,1.28
4,2 CHEAP CARS GROUP,5/03/2021 0:00,96.92,15.6,1.26


***2.Data Cleaning***
2.1 ensuring all variables are in the format of numeric

2.2 Since CRSP data included missing value. For RET, they are -66, -77，-88，-99. We should drop them from
    our dataset to make dataset clean

In [60]:
df['date'] = pd.to_datetime(df['Date'])
df['TOT RETURN IND'] = pd.to_numeric(df['TOT RETURN IND'], errors='coerce')
df['RET'] = df['TOT RETURN IND'].pct_change()
df['PRC'] = pd.to_numeric(df['UNADJUSTED PRICE'], errors='coerce')
df['VOL'] = pd.to_numeric(df['TURNOVER BY VOLUME'], errors='coerce')

df['VOL']=df['VOL']*1000


df = df[df['PRC'] > 1]
df = df[df['VOL'] > 10000]


df = df.drop(columns=['Date'])
df = df.drop(columns=['TOT RETURN IND'])
df = df.drop(columns=['UNADJUSTED PRICE'])
df = df.drop(columns=['TURNOVER BY VOLUME'])

df.head()

C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2930268426.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['Date'])


,Stock_Name,date,RET,PRC,VOL
0,2 CHEAP CARS GROUP,2021-02-25,NaN,1.30,20000.0
4,2 CHEAP CARS GROUP,2021-03-05,-0.015641,1.26,15600.0
29,2 CHEAP CARS GROUP,2021-04-13,0.000000,1.05,11900.0
56,2 CHEAP CARS GROUP,2021-05-24,0.050052,1.05,106300.0
58,2 CHEAP CARS GROUP,2021-05-26,0.000000,1.05,25400.0


***2.obtain Monthly Amihud***

In [61]:
df['DOLLAR_VOL'] = df['VOL'] * df['PRC'].abs()

df = df[df['DOLLAR_VOL'] > 0] ##only keep dollar_vol>0 in the dataset

df['amihud_daily'] = df['RET'].abs() / df['DOLLAR_VOL']*(10**6)
df.head()

,Stock_Name,date,RET,PRC,VOL,DOLLAR_VOL,amihud_daily
0,2 CHEAP CARS GROUP,2021-02-25,NaN,1.30,20000.0,26000.0,NaN
4,2 CHEAP CARS GROUP,2021-03-05,-0.015641,1.26,15600.0,19656.0,0.795730
29,2 CHEAP CARS GROUP,2021-04-13,0.000000,1.05,11900.0,12495.0,0.000000
56,2 CHEAP CARS GROUP,2021-05-24,0.050052,1.05,106300.0,111615.0,0.448434
58,2 CHEAP CARS GROUP,2021-05-26,0.000000,1.05,25400.0,26670.0,0.000000


***4.Groupby and obtain monthly Amihud Illiquidity for each stock***

In [62]:
df['month'] = df['date'].dt.to_period('M')


df = df[df['DOLLAR_VOL'] > 0]###only keep dollar volume rows   abs_ret_div_dollarvol

# calculate Amihud illiquidity group by month and permno
amihud_illiq = df.groupby(['Stock_Name', 'month'])['amihud_daily'].mean().reset_index()
amihud_illiq.rename(columns={'amihud_daily': 'Amihud_Illiquidity'}, inplace=True)
amihud_illiq.head(5)

,Stock_Name,month,Amihud_Illiquidity
0,2 CHEAP CARS GROUP,2021-02,NaN
1,2 CHEAP CARS GROUP,2021-03,0.795730
2,2 CHEAP CARS GROUP,2021-04,0.000000
3,2 CHEAP CARS GROUP,2021-05,0.692214
4,2 CHEAP CARS GROUP,2021-06,0.679169


***5. Calculating monthly Return from daily return***

In [63]:
monthR = df.groupby(['Stock_Name', 'month'])['RET'].mean().reset_index()
monthR.rename(columns={'RET': 'Month_RET'}, inplace=True)
monthR.head(5)

,Stock_Name,month,Month_RET
0,2 CHEAP CARS GROUP,2021-02,NaN
1,2 CHEAP CARS GROUP,2021-03,-0.015641
2,2 CHEAP CARS GROUP,2021-04,0.000000
3,2 CHEAP CARS GROUP,2021-05,0.031192
4,2 CHEAP CARS GROUP,2021-06,0.012014


***6.merge RET to Amihud and Quintile, groupby month and permno***

In [64]:
merged_df_Amihud_MRET = pd.merge(monthR, amihud_illiq, on=[ 'month','Stock_Name'], how='inner')
merged_df_Amihud_MRET.head(5)

,Stock_Name,month,Month_RET,Amihud_Illiquidity
0,2 CHEAP CARS GROUP,2021-02,NaN,NaN
1,2 CHEAP CARS GROUP,2021-03,-0.015641,0.795730
2,2 CHEAP CARS GROUP,2021-04,0.000000,0.000000
3,2 CHEAP CARS GROUP,2021-05,0.031192,0.692214
4,2 CHEAP CARS GROUP,2021-06,0.012014,0.679169


***7.Export to CSV***

In [65]:
merged_df_Amihud_MRET.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\20182019MonthRETAmihud.csv', index=False)

***8.Import with monthly 20182019MonthRETAmihud.csv with monthly***
This file includes montly volume and prc and share outstanding  (MonthPRC-SHAROUT.csv)

In [66]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\NZDATA-MonthData.csv'
df_MPS = pd.read_csv(file_path)

#set date
df_MPS['date'] = pd.to_datetime(df_MPS['Date'], dayfirst=True)
df_MPS['month'] = df_MPS['date'].dt.to_period('M')
df_MPS = df_MPS.drop(columns=['Date'])
print(df_MPS)

         Stock_Name  MARKET CAPITALIZATION  TOT RETURN IND       date    month
0      2 CHEAP CARS                52843.0             NaN 2021-01-01  2021-01
1      2 CHEAP CARS                40088.0           75.78 2022-01-01  2022-01
2      2 CHEAP CARS                13666.0           27.59 2023-01-01  2023-01
3      2 CHEAP CARS                52843.0             NaN 2021-02-01  2021-02
4      2 CHEAP CARS                40088.0           74.93 2022-02-01  2022-02
...             ...                    ...             ...        ...      ...
63101  ZINTEL GROUP                    NaN          114.85 2019-12-01  2019-12
63102  ZINTEL GROUP                    NaN          114.85 2020-12-01  2020-12
63103  ZINTEL GROUP                    NaN          114.85 2021-12-01  2021-12
63104  ZINTEL GROUP                    NaN          114.85 2022-12-01  2022-12
63105  ZINTEL GROUP                    NaN          114.85 2023-12-01  2023-12

[63106 rows x 5 columns]


***9. merge MonthPRC-SHAROUT.csv to merged_df_Amihud_MRET**

In [67]:
merged_df_Amihud_MRET = pd.merge(merged_df_Amihud_MRET, df_MPS, on=[ 'month','Stock_Name'], how='inner')
#when both two dataframe have same moth and permno, the row will be keep
merged_df_Amihud_MRET.head(5)

,Stock_Name,month,Month_RET,Amihud_Illiquidity,MARKET CAPITALIZATION,TOT RETURN IND,date
0,AMP NZ OFFICE TRUST UNITS,2007-07,-0.002569,0.104533,NaN,100.00,2007-07-01
1,AMP NZ OFFICE TRUST UNITS,2007-08,0.001063,0.078197,NaN,96.09,2007-08-01
2,ARGOSY PROPERTY,2003-03,0.000000,0.000000,31543.0,100.00,2003-03-01
3,ARGOSY PROPERTY,2003-04,0.010016,0.490926,237743.0,100.84,2003-04-01
4,ARGOSY PROPERTY,2003-05,0.000000,0.000000,237743.0,99.83,2003-05-01


***10.  FIRM SIZE** (marketing Capitalization is firm size)

In [68]:
merged_df_Amihud_MRET['SIZE']=merged_df_Amihud_MRET['MARKET CAPITALIZATION']

merged_df_Amihud_MRET = merged_df_Amihud_MRET.drop(columns=['MARKET CAPITALIZATION'])

merged_df_Amihud_MRET.head(5)

,Stock_Name,month,Month_RET,Amihud_Illiquidity,TOT RETURN IND,date,SIZE
0,AMP NZ OFFICE TRUST UNITS,2007-07,-0.002569,0.104533,100.00,2007-07-01,NaN
1,AMP NZ OFFICE TRUST UNITS,2007-08,0.001063,0.078197,96.09,2007-08-01,NaN
2,ARGOSY PROPERTY,2003-03,0.000000,0.000000,100.00,2003-03-01,31543.0
3,ARGOSY PROPERTY,2003-04,0.010016,0.490926,100.84,2003-04-01,237743.0
4,ARGOSY PROPERTY,2003-05,0.000000,0.000000,99.83,2003-05-01,237743.0


In [69]:
print(merged_df_Amihud_MRET)

                     Stock_Name    month  Month_RET  Amihud_Illiquidity  \
0     AMP NZ OFFICE TRUST UNITS  2007-07  -0.002569            0.104533   
1     AMP NZ OFFICE TRUST UNITS  2007-08   0.001063            0.078197   
2               ARGOSY PROPERTY  2003-03   0.000000            0.000000   
3               ARGOSY PROPERTY  2003-04   0.010016            0.490926   
4               ARGOSY PROPERTY  2003-05   0.000000            0.000000   
...                         ...      ...        ...                 ...   
5314               VULCAN STEEL  2023-08   0.001035            0.065901   
5315               VULCAN STEEL  2023-09   0.001649            0.021198   
5316               VULCAN STEEL  2023-10  -0.001157            0.038152   
5317               VULCAN STEEL  2023-11   0.001614            0.048083   
5318               VULCAN STEEL  2023-12   0.002088            0.041314   

      TOT RETURN IND       date      SIZE  
0             100.00 2007-07-01       NaN  
1          

In [70]:
#drop missing
#merged_df_Amihud_MRET = merged_df_Amihud_MRET.dropna()
#print(merged_df_Amihud_MRET)

***11.import FF3 and CAPM factors***

In [71]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\NZDATA-AlphaFactors.xlsx'
df_factor = pd.read_excel(file_path)
df_factor['date'] = pd.to_datetime(df_factor['Date'])
df_factor['month'] = df_factor['date'].dt.to_period('M')
df_factor = df_factor.drop(columns=['Date'])
df_factor.head(5)

,smb,hml,RF,mkt,Mkt-RF,date,month
0,0.030127,0.003332,0.005876,-0.067563,-0.073439,2000-01-31,2000-01
1,-0.005431,-0.016740,0.005234,-0.030049,-0.035283,2000-02-29,2000-02
2,-0.075748,-0.089202,0.006501,0.052268,0.045767,2000-03-31,2000-03
3,0.003947,0.073622,0.005573,-0.020173,-0.025746,2000-04-30,2000-04
4,-0.048036,0.045267,0.005686,-0.005924,-0.011610,2000-05-31,2000-05


***12.merge FF3 and CAPM factors***

In [72]:
merged_df_Amihud_MRET = pd.merge(merged_df_Amihud_MRET, df_factor, on=[ 'month'], how='inner')
#when both two dataframe have same moth and permno, the row will be keep
merged_df_Amihud_MRET.head(5)

,Stock_Name,month,Month_RET,Amihud_Illiquidity,TOT RETURN IND,date_x,SIZE,smb,hml,RF,mkt,Mkt-RF,date_y
0,AMP NZ OFFICE TRUST UNITS,2007-07,-0.002569,0.104533,100.00,2007-07-01,NaN,-0.021838,0.006499,0.005354,-0.005245,-0.010600,2007-07-31
1,AMP NZ OFFICE TRUST UNITS,2007-08,0.001063,0.078197,96.09,2007-08-01,NaN,0.017687,0.005787,0.005606,-0.023661,-0.029267,2007-08-31
2,ARGOSY PROPERTY,2003-03,0.000000,0.000000,100.00,2003-03-01,31543.0,-0.009265,0.039162,0.005352,0.026563,0.021211,2003-03-31
3,ARGOSY PROPERTY,2003-04,0.010016,0.490926,100.84,2003-04-01,237743.0,0.040077,-0.030019,0.005051,0.041545,0.036494,2003-04-30
4,ARGOSY PROPERTY,2003-05,0.000000,0.000000,99.83,2003-05-01,237743.0,0.053718,-0.011564,0.004827,0.036074,0.031248,2003-05-31


***13.Save The dataset, we will use this dataset to do the dobule sort***

In [73]:
merged_df_Amihud_MRET.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\20182019forDoubleSort.csv', index=False)

***14. Import the dataset we just saved***

In [74]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\20182019forDoubleSort.csv'
df_DS = pd.read_csv(file_path)
df_DS.head()


,Stock_Name,month,Month_RET,Amihud_Illiquidity,TOT RETURN IND,date_x,SIZE,smb,hml,RF,mkt,Mkt-RF,date_y
0,AMP NZ OFFICE TRUST UNITS,2007-07,-0.002569,0.104533,100.00,2007-07-01,NaN,-0.021838,0.006499,0.005354,-0.005245,-0.010600,2007-07-31
1,AMP NZ OFFICE TRUST UNITS,2007-08,0.001063,0.078197,96.09,2007-08-01,NaN,0.017687,0.005787,0.005606,-0.023661,-0.029267,2007-08-31
2,ARGOSY PROPERTY,2003-03,0.000000,0.000000,100.00,2003-03-01,31543.0,-0.009265,0.039162,0.005352,0.026563,0.021211,2003-03-31
3,ARGOSY PROPERTY,2003-04,0.010016,0.490926,100.84,2003-04-01,237743.0,0.040077,-0.030019,0.005051,0.041545,0.036494,2003-04-30
4,ARGOSY PROPERTY,2003-05,0.000000,0.000000,99.83,2003-05-01,237743.0,0.053718,-0.011564,0.004827,0.036074,0.031248,2003-05-31


***15. divide stock in to 5 quintile by SIZE for each month***

In [75]:

df_DS['size_quintile'] = (
    df_DS.groupby('month')['SIZE']
    .transform(lambda x: pd.qcut(x.rank(method='first'), q=5, labels=[1, 2,3,4,5]))
)
####ranking from some to large. 1 is smallest firms, 5 is largest


#df_DS.head(5)

counts = df_DS['size_quintile'].value_counts().sort_index()
print(counts)
#you can see the nubmer of each quintile is different, becasue we calculate based on each month.
#and for each month, the nubmer of stock potentially different.



#following code to see each size group's average amihud
#avg_illiquidity_by_quintile = df_DS.groupby('size_quintile')['Amihud_Illiquidity'].mean()
#print(avg_illiquidity_by_quintile)

#SIZE rank
avg_illiquidity_by_quintile = df_DS.groupby('size_quintile')['SIZE'].mean()
print(avg_illiquidity_by_quintile)

size_quintile
1    684
2    497
3    515
4    497
5    638
Name: count, dtype: int64
size_quintile
1    1.281656e+05
2    4.184483e+05
3    1.058169e+06
4    2.686312e+06
5    7.121331e+06
Name: SIZE, dtype: float64


C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\4073433939.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_illiquidity_by_quintile = df_DS.groupby('size_quintile')['SIZE'].mean()


***16. Further,divide stock in to 5 quintile by amihud, for each size quintile***

In [76]:
df_DS['size_Amihud_quntiles'] = (
    df_DS.groupby('size_quintile')['Amihud_Illiquidity']
    .transform(lambda x: pd.qcut(x.rank(method='first'), q=5, labels=[1, 2,3,4,5]))
)
#df_DS.head()

df_DS.groupby(['size_quintile', 'size_Amihud_quntiles']).size()





C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\31960022.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_DS.groupby('size_quintile')['Amihud_Illiquidity']
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\31960022.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_DS.groupby(['size_quintile', 'size_Amihud_quntiles']).size()


size_quintile  size_Amihud_quntiles
1              1                       137
               2                       137
               3                       136
               4                       137
               5                       137
2              1                       100
               2                        99
               3                        99
               4                        99
               5                       100
3              1                       103
               2                       103
               3                       103
               4                       103
               5                       103
4              1                       100
               2                        99
               3                        99
               4                        99
               5                       100
5              1                       128
               2                       127
               3  

***17.show amihud illiquidity in all 25 portfolio***
we can tell, in each size quintile, amihud quintile=1, highest liquidity group. amihud

In [77]:
#df_DS.head()  size_Amihud_quntiles
avg_Amihud = df_DS.groupby(['size_quintile','size_Amihud_quntiles'])['Amihud_Illiquidity'].mean()
print(avg_Amihud)

size_quintile  size_Amihud_quntiles
1              1                       0.023451
               2                       0.135220
               3                       0.286655
               4                       0.525102
               5                       1.686534
2              1                       0.013137
               2                       0.040383
               3                       0.070007
               4                       0.125735
               5                       0.448217
3              1                       0.003968
               2                       0.008591
               3                       0.015692
               4                       0.025317
               5                       0.083592
4              1                       0.001424
               2                       0.002947
               3                       0.004975
               4                       0.008717
               5                       0.019425
5   

C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\1805760053.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_Amihud = df_DS.groupby(['size_quintile','size_Amihud_quntiles'])['Amihud_Illiquidity'].mean()



***18. obtain eqaul weight return for all 25 portfolio***
#df_DS.head()

In [78]:

portfolio_ret = (
    df_DS
    .groupby([ 'size_quintile', 'size_Amihud_quntiles'])['Month_RET']
    .mean()
    .reset_index()
    .rename(columns={'simple_return': 'port_ret'})
)

print(portfolio_ret)

   size_quintile size_Amihud_quntiles  Month_RET
0              1                    1  -0.001175
1              1                    2   0.001660
2              1                    3   0.000914
3              1                    4   0.000212
4              1                    5   0.010717
5              2                    1   0.000131
6              2                    2   0.001727
7              2                    3  -0.000224
8              2                    4   0.001612
9              2                    5   0.000689
10             3                    1   0.000630
11             3                    2   0.001326
12             3                    3   0.000426
13             3                    4   0.001092
14             3                    5  -0.000237
15             4                    1  -0.002245
16             4                    2   0.000328
17             4                    3   0.000721
18             4                    4   0.000755
19             4    

C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\3672364723.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([ 'size_quintile', 'size_Amihud_quntiles'])['Month_RET']


***calculate Q5-Q1 for eqaul weight return***

In [79]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Step 1: Equal‑weighted monthly return for every (size, Amihud) portfolio
port_ret_monthly = (
    df_DS
    .groupby(["month", "size_quintile", "size_Amihud_quntiles"])["Month_RET"]
    .mean()
    .reset_index()
)

# Step 2: For every size‑quintile, build the long–short series (Q1 − Q5)
ls_list = []

for s, g in port_ret_monthly.groupby("size_quintile"):
    wide = g.pivot(index="month",
                   columns="size_Amihud_quntiles",
                   values="Month_RET")
    wide = wide.dropna(subset=[1, 5])
    wide["LS"] = wide[5] - wide[1]
    wide["size_quintile"] = s
    wide = wide.reset_index() 
    ls_list.append(wide[["month", "size_quintile", "LS"]].reset_index(drop=True))

ls_df = pd.concat(ls_list, ignore_index=True)

# Step 3: Statistical significance test (mean, t‑statistic, etc.)
summary = []

for s, g in ls_df.groupby("size_quintile"):
    mu = g["LS"].mean()
    ann_mu = mu * 12  # annualized return

    res = sm.OLS(g["LS"], np.ones(len(g))).fit(
        cov_type="HAC", cov_kwds={"maxlags": 11}
    )

    summary.append({
        "size_quintile": s,
        "mean_monthly": mu,
        
        "t_stat": res.tvalues[0],
        
    })

summary_df = pd.DataFrame(summary)
print(summary_df)


   size_quintile  mean_monthly    t_stat
0              1      0.004943  0.624086
1              2     -0.006658 -6.119722
2              3      0.001582  2.405421
3              4      0.000320  0.858839
4              5      0.000178  0.414955


C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2733258204.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["month", "size_quintile", "size_Amihud_quntiles"])["Month_RET"]
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2733258204.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for s, g in port_ret_monthly.groupby("size_quintile"):
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2733258204.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To

***19.CAPM return***

In [80]:

import pandas as pd
import statsmodels.api as sm



# Variable basic processing
df_DS[["Mkt-RF", "RF"]] = df_DS[["Mkt-RF", "RF"]] 
df_DS["excess_return"] = df_DS["Month_RET"] - df_DS["RF"]  
df_DS["month"] = pd.to_datetime(df_DS["month"])

#calculate excess reutrn at t+1
df_DS = df_DS.sort_values(["Stock_Name", "month"]) #ensure time-series 
df_DS["excess_return_t1"] = df_DS.groupby("Stock_Name")["excess_return"].shift(-1)  #create excess return at t+1


#Construct t+1 equal-weighted porfolio return

portfolio_returns = (
    df_DS.groupby(["month", "size_quintile", "size_Amihud_quntiles"])["excess_return_t1"]
         .mean()
         .reset_index()
         .rename(columns={"excess_return_t1": "port_excess_ret_t1"}) # construct and rename the portfolio excess return
)

###create the dataset that only have ensential variables
factors = df_DS[["month", "Mkt-RF"]].drop_duplicates()
df_port = pd.merge(portfolio_returns, factors, on="month", how="left")


# Capm regression on each portfolio size × Amihud， 25 portfolio in total
results = []

grouped = df_port.groupby(["size_quintile", "size_Amihud_quntiles"])
#Group the entire df_port data dataset by size_quintile and size_Amihud_quntiles.


# the next sentece use the group key above, size_q is "size_quintile",illiq_q "size_Amihud_quntiles"
for (size_q, illiq_q), group in grouped: # iteration each group of the grouped ("size_quintile" * "size_Amihud_quntiles")
    group = group.sort_values("month") #Sort the current set of data by "month" from small to large.
    group = group.dropna(subset=["port_excess_ret_t1", "Mkt-RF"])


    X = sm.add_constant(group["Mkt-RF"])
    y = group["port_excess_ret_t1"]
    
    if len(group) >= 12:  #at least 12 obersvations
        res = sm.OLS(y, X).fit()
        results.append({
            "size_quintile": size_q,
            "Amihud_quintile": illiq_q,
            "alpha": res.params["const"],
            "alpha_tstat": res.tvalues["const"],
            "beta": res.params["Mkt-RF"],
            "n_obs": len(group)
        })
        #The purpose of this section is to save the
        # regression results of the current combination in the form of a dictionary into the results list
    else:
        results.append({
            "size_quintile": size_q,
            "Amihud_quintile": illiq_q,
            "alpha": None,
            "alpha_tstat": None,
            "beta": None,
           # "n_obs": len(group)
        })


# print as dataframe, and sorting by each portfolio
alpha_df = pd.DataFrame(results).sort_values(["size_quintile", "Amihud_quintile"])
print(alpha_df)


    size_quintile  Amihud_quintile     alpha  alpha_tstat      beta  n_obs
0               1                1 -0.002832    -1.672542  0.001276    119
1               1                2 -0.002155    -2.388261 -0.013643    122
2               1                3 -0.002521    -2.420450 -0.018597    121
3               1                4 -0.002193    -1.234111  0.109124    115
4               1                5 -0.002663    -1.051690  0.045759    118
5               2                1 -0.002966    -7.731700  0.003533     95
6               2                2 -0.001543    -2.289641  0.002007     85
7               2                3 -0.002510    -4.472163 -0.001471     84
8               2                4 -0.002865    -4.746314  0.044568     90
9               2                5 -0.000559    -0.316301  0.076285     86
10              3                1 -0.002119    -5.577066  0.010574     89
11              3                2 -0.003059   -11.718914  0.014957     94
12              3        

C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\3613219125.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_DS.groupby(["month", "size_quintile", "size_Amihud_quntiles"])["excess_return_t1"]
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\3613219125.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_port.groupby(["size_quintile", "size_Amihud_quntiles"])


****20.Test whether the difference in the returns/alphabetween the bottom and top quintile in each size portfolio is economically and statistically significant***
R_it - R_ft = α_i + β_i (R_mt - R_ft) + ε_t

In [81]:
import numpy as np
import statsmodels.api as sm

# prepare market factors, they will be used in further regression
factors = (
    df_DS[["month", "Mkt-RF"]]
    .drop_duplicates()         # only keep one row in each month. because they are same each month
)



#calculate each size group's Q1‑Q5 (high liquidity - low liquidity) monthly excess return
ls_dfs = []                    # we create this for saving 5 size portfolios' results

for s in range(1, 6):          # size_quintile = 1,2,3,4,5, this is an iteration
    # extract current size group;s data
    df_size = df_port[df_port["size_quintile"] == s].copy()



    # 2‑2 Expand the Amihud quintiles into columns by month (1…5)
    # Pivot_table is more stable than pivot here, and missing values ​​are automatically NaN
    wide = df_size.pivot_table(
        index   = "month",
        columns = "size_Amihud_quntiles",
        values  = "port_excess_ret_t1"
    )


    # 2‑3  Long‑Short：Q1 (high liquidity) − Q5 (low liquidity)
    wide["LS"] = wide[1] - wide[5]

    


    wide = (
        wide[["LS"]]              # only keep LS column
        .dropna()                 # drip missing
        .reset_index()            # make month not index, just column
    )
    wide["size_quintile"] = s    # 标记 size 组

    ls_dfs.append(wide)


ls_df = pd.concat(ls_dfs, ignore_index=True)

# 2‑6 merge market fators
ls_df = ls_df.merge(factors, on="month", how="left").dropna()

# --------------------------------------------------
# 3. summary for each size group
# --------------------------------------------------
summary_rows = []

for s in range(1, 6):
    sub = ls_df[ls_df["size_quintile"] == s].copy()

    # ---mean average
    mean_monthly = sub["LS"].mean()
    #mean_annual  = mean_monthly * 12




    # --- regression- t-test
    res_mean = sm.OLS(sub["LS"], np.ones(len(sub))).fit(
        cov_type = "HAC", cov_kwds = {"maxlags": 11}
    )

    # --- 3‑3 statistical significance (ii): CAPM regression  → α and t‑stat
    X = sm.add_constant(sub["Mkt-RF"])
    res_capm = sm.OLS(sub["LS"], X).fit(
        cov_type = "HAC", cov_kwds = {"maxlags": 11}
    )

    # --- 3‑4 collect result
    summary_rows.append({
        "size_quintile" : s,
        "mean_monthly"  : mean_monthly,  # Q1 (high liquidity) − Q5 (low liquidity) monthly average return
        "mean_tstat"    : res_mean.tvalues[0],  # is mean_monthly statistically significant
        "alpha_capm"    : res_capm.params["const"],  #Q1 (high liquidity) − Q5 (low liquidity) monthly alpha capm
        "alpha_tstat"   : res_capm.tvalues["const"], #t-value of CAPM ALPHA
        
    })

# 3‑5 print table
ls_summary = (
    pd.DataFrame(summary_rows)
    .round(4)                 # 保留 4 位小数，够用且整洁
    .set_index("size_quintile")
)

print(ls_summary)




               mean_monthly  mean_tstat  alpha_capm  alpha_tstat
size_quintile                                                   
1                    0.0019      0.5460      0.0020       0.5516
2                    0.0035      1.7739      0.0043       3.8686
3                   -0.0014     -1.6529     -0.0003      -0.4813
4                   -0.0007     -1.7207     -0.0007      -1.9692
5                   -0.0013     -2.8927     -0.0012      -3.3731


C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2276887399.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide = df_size.pivot_table(
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2276887399.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide = df_size.pivot_table(
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2276887399.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide = df_size.pivot_table(
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\2276887399.py:23: Fu

***FF3 regression***

In [82]:

import pandas as pd
import statsmodels.api as sm


factor_cols = ["Mkt-RF", "smb", "hml", "RF"]     
#df_DS[factor_cols] = df_DS[factor_cols] / 100
df_DS["Mkt-RF"] = df_DS["Mkt-RF"] 
df_DS["RF"] = df_DS["RF"] 

# calculating excess return
df_DS["M-RET"] = df_DS["Month_RET"] 
df_DS["excess_return"] = df_DS["M-RET"] - df_DS["RF"]

#
df_DS["month"] = pd.to_datetime(df_DS["month"])

#excess return t+1
df_DS = df_DS.sort_values(["Stock_Name", "month"])
df_DS["excess_return_t1"] = (
    df_DS.groupby("Stock_Name")["excess_return"].shift(-1)
)

# construct t+1 excess return portfolio
portfolio_returns = (
    df_DS
    .groupby(["month", "size_quintile", "size_Amihud_quntiles"])["excess_return_t1"]
    .mean()
    .reset_index()
    .rename(columns={"excess_return_t1": "port_excess_ret_t1"})
)

# 
# merge fama frence
factors = (
    df_DS[["month", "Mkt-RF", "smb", "hml"]]
    .drop_duplicates()
)

df_port = pd.merge(portfolio_returns, factors, on="month", how="left")

# regression-for both 25 portfolio
results = []

grouped = df_port.groupby(["size_quintile", "size_Amihud_quntiles"])

for (size_q, illiq_q), group in grouped:
    
    group = (
        group.sort_values("month")
             .dropna(subset=["port_excess_ret_t1", "Mkt-RF", "smb", "hml"])
    )

    
    X = sm.add_constant(group[["Mkt-RF", "smb", "hml"]])
    y = group["port_excess_ret_t1"]

    # at least we have 12 periods
    if len(group) >= 12:
        
        res = sm.OLS(y, X).fit(
            cov_type="HAC", cov_kwds={"maxlags":11}
        )
        results.append({
            "size_quintile":      size_q,
            "Amihud_quintile":    illiq_q,
            "ff3_alpha":              res.params["const"],
            "alpha_tstat":        res.tvalues["const"],
           
            "n_obs":              len(group)
        })
    else:
        
        results.append({
            "size_quintile":      size_q,
            "Amihud_quintile":    illiq_q,
            "ff3_alpha":              None,
            "alpha_tstat":        None,
            
            "n_obs":              len(group)
        })

# print result
ff3_alpha_df = (
    pd.DataFrame(results)
      .sort_values(["size_quintile", "Amihud_quintile"])
      .reset_index(drop=True)
)

print(ff3_alpha_df)


C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\1580461633.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["month", "size_quintile", "size_Amihud_quntiles"])["excess_return_t1"]
C:\Users\yumi7517\AppData\Local\Temp\ipykernel_19264\1580461633.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_port.groupby(["size_quintile", "size_Amihud_quntiles"])


    size_quintile  Amihud_quintile  ff3_alpha  alpha_tstat  n_obs
0               1                1  -0.002960    -1.692425    119
1               1                2  -0.002306    -2.441081    122
2               1                3  -0.002371    -1.835979    121
3               1                4  -0.002198    -0.877971    115
4               1                5  -0.002594    -1.455063    118
5               2                1  -0.002842    -5.403116     95
6               2                2  -0.001395    -1.901267     85
7               2                3  -0.002390    -3.400795     84
8               2                4  -0.002897    -3.798133     90
9               2                5  -0.000536    -0.404151     86
10              3                1  -0.002255    -6.181863     89
11              3                2  -0.003106    -7.925091     94
12              3                3  -0.002758    -7.802777     93
13              3                4  -0.002985    -4.255391     82
14        

In [83]:
#ff3_alpha_df.to_csv(r'D:\DATA for CLASS\test\ff3_alpha_df.csv', index=False)

***fama french Q5-Q1***

In [84]:
ls_list = []

for s in range(1, 6):                     # size_quintile 1…5
    tmp = df_port[df_port["size_quintile"] == s]

    # 5 illiquidity portfolio in 5 column
    tmp_wide = tmp.pivot(index="month",
                         columns="size_Amihud_quntiles",
                         values="port_excess_ret_t1")

   
    tmp_wide = tmp_wide.dropna(subset=[1, 5])

    tmp_wide["LS"] = tmp_wide[5] - tmp_wide[1]   
    tmp_wide["size_quintile"] = s
    ls_list.append(tmp_wide[["size_quintile", "LS"]].reset_index())

ls_df = pd.concat(ls_list, ignore_index=True)

#merge factors
ls_df = ls_df.merge(factors, on="month", how="left").dropna()

# ff3 regression for each q5-q1
ls_results = []

for s, g in ls_df.groupby("size_quintile"):
    X = sm.add_constant(g[["Mkt-RF", "smb", "hml"]])
    y = g["LS"]

    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 11})

    ls_results.append({
        "size_quintile": s,
        "LS_alpha"     : res.params["const"],
        "alpha_tstat"  : res.tvalues["const"],
       
    })

ls_alpha_df = pd.DataFrame(ls_results)

# print result
print(ls_alpha_df)

   size_quintile  LS_alpha  alpha_tstat
0              1  0.000949     0.377593
1              2 -0.020247   -10.568927
2              3  0.000057     0.081176
3              4  0.001172     2.575571
4              5  0.001161     3.787831
